In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.iv import IVGMM
from linearmodels.iv import IV2SLS
from statsmodels.api import add_constant

In [2]:
np.seterr(divide = 'ignore')

# all
df = pd.read_csv('data/output/bonds/herding_bondMarket_None_2002-2022.csv')
df = df.dropna()
# add constant
df = add_constant(df, has_constant='add')

# retail
df_ret = pd.read_csv('data/output/bonds/herding_bondMarket_ret_2002-2022.csv')
df_ret = df_ret.dropna()
# add constant
df_ret = add_constant(df_ret, has_constant='add')

# retail
df_int = pd.read_csv('data/output/bonds/herding_bondMarket_int_2002-2022.csv')
df_int = df_int.dropna()
# add constant
df_int = add_constant(df_int, has_constant='add')

df.head()

,const,Date,Rm,AbsoluteRm,SquaredRm,Csad,LeftTail,RightTail
0,1.0,2002-07-31,-0.002184,0.002184,0.000005,0.072513,0,0
1,1.0,2002-08-31,0.034135,0.034135,0.001165,1.890931,0,1
2,1.0,2002-09-30,0.003122,0.003122,0.000010,0.987137,0,0
3,1.0,2002-10-31,-0.007174,0.007174,0.000051,0.820465,0,0
4,1.0,2002-11-30,0.043976,0.043976,0.001934,3.016087,0,1


In [3]:
# period splitting
df = df[(df['Date'] >= '2002-01-01') & (df['Date'] < '2023-01-01')]
df_ret = df_ret[(df_ret['Date'] >= '2002-01-01') & (df_ret['Date'] < '2023-01-01')]
df_int = df_int[(df_int['Date'] >= '2002-01-01') & (df_int['Date'] < '2023-01-01')]

## Model 1: All

#### OLS

In [4]:
x1 = df[
    ['Rm', 'AbsoluteRm', 'SquaredRm']
]
y1 = df['Csad']

x1 = sm.add_constant(x1)

model1 = sm.OLS(y1, x1).fit()

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                   Csad   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     237.1
Date:                Tue, 22 Aug 2023   Prob (F-statistic):           2.53e-71
Time:                        01:48:20   Log-Likelihood:                -194.87
No. Observations:                 243   AIC:                             397.7
Df Residuals:                     239   BIC:                             411.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0065      0.062     -0.104      0.9

#### TSNL

In [5]:
res_ols_v1 = IVGMM(
    df.Csad, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v1)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   Csad   R-squared:                      0.7485
Estimator:                     IV-GMM   Adj. R-squared:                 0.7453
No. Observations:                 243   F-statistic:                    723.05
Date:                Tue, Aug 22 2023   P-value (F-stat)                0.0000
Time:                        01:48:20   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             0.2472     2.1170     0.1168     0.9071     -3.9020      4.3964
AbsoluteRm     97.256     6.2971     15.445     0.00

## Model 2: Retail

#### OLS

In [6]:
x1 = df_ret[
    ['Rm', 'AbsoluteRm', 'SquaredRm']
]
y1 = df_ret['Csad']

x1 = sm.add_constant(x1)

model2 = sm.OLS(y1, x1).fit()

print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                   Csad   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     279.1
Date:                Tue, 22 Aug 2023   Prob (F-statistic):           1.24e-77
Time:                        01:48:21   Log-Likelihood:                -178.67
No. Observations:                 242   AIC:                             365.3
Df Residuals:                     238   BIC:                             379.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1715      0.061      2.809      0.0

#### TSNL

In [7]:
res_ols_v2 = IVGMM(
    df_ret.Csad, df_ret[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v2)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   Csad   R-squared:                      0.7787
Estimator:                     IV-GMM   Adj. R-squared:                 0.7759
No. Observations:                 242   F-statistic:                    851.47
Date:                Tue, Aug 22 2023   P-value (F-stat)                0.0000
Time:                        01:48:21   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             4.1637     4.0854     1.0192     0.3081     -3.8436      12.171
AbsoluteRm     114.22     11.647     9.8069     0.00

## Model 3: Institunional|

#### OLS

In [8]:
x1 = df_int[
    ['Rm', 'AbsoluteRm', 'SquaredRm']
]
y1 = df_int['Csad']

x1 = sm.add_constant(x1)

model3 = sm.OLS(y1, x1).fit()

print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                   Csad   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     237.0
Date:                Tue, 22 Aug 2023   Prob (F-statistic):           2.56e-71
Time:                        01:48:21   Log-Likelihood:                -194.88
No. Observations:                 243   AIC:                             397.8
Df Residuals:                     239   BIC:                             411.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0065      0.062     -0.104      0.9

#### TSNL

In [9]:
res_ols_v3 = IVGMM(
    df_int.Csad, df_int[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v3)

                          IV-GMM Estimation Summary                           
Dep. Variable:                   Csad   R-squared:                      0.7484
Estimator:                     IV-GMM   Adj. R-squared:                 0.7453
No. Observations:                 243   F-statistic:                    722.94
Date:                Tue, Aug 22 2023   P-value (F-stat)                0.0000
Time:                        01:48:21   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             0.2474     2.1170     0.1169     0.9070     -3.9018      4.3966
AbsoluteRm     97.255     6.2967     15.445     0.00